In [ ]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON
from pathlib import Path
import multiprocessing as mp  
from multiprocessing import  Pool 
from rdflib import Graph
import os

In [ ]:
import urllib.request

with urllib.request.urlopen('http://172.18.0.2:8890/sparql') as response:
    print(response.read())

In [ ]:
def fetchBatch(sparql, values_arr, entity):
    
    results = Graph()
    
    for file in os.listdir("queries"):
        if file.startswith("chembl-to-bioschema-"+entity):
            
            queryString = Path("queries/"+file).read_text()
            queryString = queryString.replace("@_MOLS_@", values_arr)
    
            sparql.setQuery(queryString)
            results += sparql.query().convert()
    
    return results

In [ ]:
def getConstructKgForRange(offset, sparql, sparql2, entity):

    try:
        
        queryStringGetAllIds = "Select ?iri where { Select ?iri where { ?iri a <http://rdf.ebi.ac.uk/terms/chembl#"+entity+"> } order by ?iri } limit 100000 offset "

        offset = offset * 100000

        queryStringGetAllIds = queryStringGetAllIds.strip() + " " +str(offset)

        sparql.setQuery(queryStringGetAllIds)
        results = sparql.query()

        j = 1

        values_arr = ""

        output = Graph()

        for result in results.bindings:

            if(j % 20 == 0):

                values_arr += "<"+str(result["iri"].value)+"> "

                output += fetchBatch(sparql2, values_arr, entity)

                values_arr = ""
                
                print("batch " + str(j))

            else:
                values_arr += "<"+str(result["iri"].value)+"> "

            j = j + 1

        if(values_arr != ""):
            output += fetchBatch(sparql2, values_arr, entity)

        with open("json-ld-output/chembl-"+entity+"-"+str(offset)+".json", "w") as file:
            file.write(output.serialize(format='json-ld', indent=0))

        print("Got JSON-LD for batch " + str(j))
    
    except Exception:
        getConstructKgForRange(offset, sparql, sparql2, entity)

In [ ]:
sparql = SPARQLWrapper2("http://172.18.0.2:8890/sparql")
sparql2 = SPARQLWrapper("http://172.18.0.2:8890/sparql")

for i in range(0,20):
    getConstructKgForRange(i, sparql, sparql2, "SmallMolecule")

In [ ]:
sparql = SPARQLWrapper2("http://172.18.0.2:8890/sparql")
sparql2 = SPARQLWrapper("http://172.18.0.2:8890/sparql")

getConstructKgForRange(0, sparql, sparql2, "SingleProtein")